## joda_01_kehitysympäristö
Jupyter Notebook (ver. 5.7.8) ympäristö Anaconda (ver. 1.9.7) jakeluna omalle PC:lle asennettuna. <br>
Asennus onnistuu https://www.anaconda.com/distribution/ sivuilta valitsemalla oikea käyttöjärjestelmä ja haluttu Python versio.<br>

Editorina Microsoft Visual Studio Code (ver. 1.40.1) PC:lle asennettuna. <br>
Asennus onnistuu https://code.visualstudio.com/download sivuilta valitsemalla oikea käyttöjärjestelmä. <br>

GitHub toimii julkaisualustana sivulla https://github.com/mvehmane/airbnb_munich <br>
GitHub tili on tullut joskus aikaisemmin aktivoitua vaikka en sitä olekaan varsinaisesti käyttänyt. <br>
GitHub on käytettävissä sekä web että desktop ympäristössä. <br>
Asennan myös desktop version harjoittelua varten. <br>
Desktop asennus https://desktop.github.com/ <br>

Näillä työkaluilla yritetään pärjätä ja tarvittaessa asennetaan lisää harjoituksen tekemiseksi.


### kirjastojen haku
asennetaan kirjastot yhdessä solussa: <br>
pandas - dataframes <br>
numpy - math <br>
sklearn - machine learning <br>
matplotlib - numerical plotting <br>
collections _ <br>
scipy - numerical plotting<br>
seaborn - data visualization <br>
plotly - data analytics and visualization <br>
dash - analytical web applications <br>








In [1]:
#kirjastojen asennus
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
import scipy
import seaborn as sns


## joda_02_datan_kerääminen
Käytetään insideairbnb sivuston dataa kohdekaupunkina Munich, käytettävissä on kolme datapakettia. <br>
Inside Airbnb sivusto ei kerro tarkemmin datan ominaisuuksista, joten on itse arvioitava eri tietojen hyödyllisyyttä ja käyttökelpoisuutta. <br>
Haetaan kaikki kolme datapakettia: <br>
1. Detailed Listings data for Munich <br>
2. Detailed Calendar Data for listings in Munich <br>
3. Detailed Review Data for listings in Munich <br>




In [2]:
listings_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/listings.csv.gz'

# read listings file into a dataframe
df_listings = pd.read_csv(listings_file, compression='gzip')
df_listings.to_csv('listings.csv')
# tutkitaan datasettiä sarakenimien avulla
pd.options.display.max_rows=110
pd.Series(df_listings.columns)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (43,61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0                                                id
1                                       listing_url
2                                         scrape_id
3                                      last_scraped
4                                              name
5                                           summary
6                                             space
7                                       description
8                               experiences_offered
9                             neighborhood_overview
10                                            notes
11                                          transit
12                                           access
13                                      interaction
14                                      house_rules
15                                    thumbnail_url
16                                       medium_url
17                                      picture_url
18                                   xl_picture_url
19          

In [3]:
# read calendar file into a dataframe
calendar_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/calendar.csv.gz'
df_calendar = pd.read_csv(calendar_file, compression='gzip')
df_calendar.to_csv('calendar.csv')
# sarakkeiden listaus
df_calendar.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [4]:
# read reviews file into a dataframe
reviews_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/reviews.csv.gz'
df_reviews = pd.read_csv(reviews_file, compression='gzip')
df_reviews.to_csv('reviews.csv')
# sarakkeiden listaus
df_reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [5]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,36720,12723661,2014-05-09,11840468,Mikhail,I would like to thank Gabriella as her apartme...
1,36720,13147830,2014-05-20,5466213,Kim,Gabriela's place was absolutely fantastic. It...
2,36720,16302574,2014-07-25,2062882,Juan R.,"Quiet place, open to a courtyard, with all the..."
3,36720,16428874,2014-07-27,1225618,David,The best Airbnb expeierence I've had. The apar...
4,36720,19478358,2014-09-13,13977301,Cal,"All first rate ! Beautiful apartment, comforta..."


In [6]:
# yhdistetään df_listings ja df.reviews datasetit kohteen id:llä ja luodaan listings_reviews.csv

df_listings_reviews = df_listings.merge(df_reviews, left_on='id', right_on='listing_id')
df_listings_reviews.to_csv('listings_reviews.csv')

In [7]:
df_listings_reviews.head(3)


,id_x,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,listing_id,id_y,date,reviewer_id,reviewer_name,comments
0,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,12723661,2014-05-09,11840468,Mikhail,I would like to thank Gabriella as her apartme...
1,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,13147830,2014-05-20,5466213,Kim,Gabriela's place was absolutely fantastic. It...
2,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,16302574,2014-07-25,2062882,Juan R.,"Quiet place, open to a courtyard, with all the..."


## joda_03_datan_jalostaminen
Hahmotellaan datasettien sisältöä:<br>
-koko<br>
-tietotyyppi<br>
-NaN (not a number) arvojen etsiminen<br>
Korjataan dataa tarvittaessa

Datasettien hahmottelua df.shape metodilla<br>
(rivimäärä, sarakemäärä)


In [8]:
df_reviews.shape


(170005, 6)

In [9]:
print(df_listings.shape)

(12357, 106)


In [10]:
print(df_calendar.shape)

(4510305, 7)


In [11]:
df_listings_reviews.shape

(170005, 112)

Datasettien hahmottelua df.info metodilla

In [12]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12357 entries, 0 to 12356
Columns: 106 entries, id to reviews_per_month
dtypes: float64(24), int64(21), object(61)
memory usage: 10.0+ MB


In [13]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510305 entries, 0 to 4510304
Data columns (total 7 columns):
listing_id        int64
date              object
available         object
price             object
adjusted_price    object
minimum_nights    int64
maximum_nights    int64
dtypes: int64(3), object(4)
memory usage: 240.9+ MB


In [14]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170005 entries, 0 to 170004
Data columns (total 6 columns):
listing_id       170005 non-null int64
id               170005 non-null int64
date             170005 non-null object
reviewer_id      170005 non-null int64
reviewer_name    170005 non-null object
comments         169940 non-null object
dtypes: int64(3), object(3)
memory usage: 7.8+ MB


In [15]:
df_listings_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170005 entries, 0 to 170004
Columns: 112 entries, id_x to comments
dtypes: float64(24), int64(24), object(64)
memory usage: 146.6+ MB


Alustavasti näyttää siltä, että dataa on riittävästi.<br>
Tässä vaiheessa myös päätän käyttää em. dataseteistä kahta:<br>
df_listings<br>
df_listings_reviews<br>

Molemmissa seteissä on yli 100 saraketta dataa ja koska Inside Airbnb sivulta en onnistunut löytämään datalle kuvauksia päätän seurata https://mapr.com/blog/predicting-airbnb-listing-prices-scikit-learn-and-apache-spark/ sivun esimerkkiä pienin muutoksin df_listings datasetin alustavaan muokkaamiseen.


In [16]:
cols1 = ['price',
        'accommodates',
        'bedrooms',
        'beds',
        'neighbourhood_cleansed',
        'room_type',
        'reviews_per_month',
        'number_of_reviews',
        'review_scores_rating',
        'latitude',
        'longitude'
        ]

df_listings_cleansed = pd.read_csv('listings.csv', usecols=cols1)
#tarkistetaan tulos
df_listings_cleansed.head()

,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month
0,Ludwigsvorstadt-Isarvorstadt,48.13057,11.56929,Entire home/apt,2,1.0,1.0,$95.00,25,98.0,0.38
1,Hadern,48.11476,11.48782,Entire home/apt,2,1.0,1.0,$80.00,128,97.0,1.32
2,Berg am Laim,48.11923,11.63726,Entire home/apt,5,1.0,3.0,$95.00,48,95.0,0.48
3,Maxvorstadt,48.15198,11.56486,Entire home/apt,4,1.0,1.0,$120.00,82,97.0,0.81
4,Pasing-Obermenzing,48.13898,11.46612,Entire home/apt,3,1.0,1.0,$55.00,30,93.0,0.30


Muokataan price tiedot numeeriseen muotoon, poistetaan $ merkki

In [17]:

df_listings_cleansed['price'] = df_listings_cleansed['price'].replace('[\$,)]','',  \
        regex=True).replace('[(]','-', regex=True).astype(float)

In [18]:
df_listings_cleansed.head()

,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month
0,Ludwigsvorstadt-Isarvorstadt,48.13057,11.56929,Entire home/apt,2,1.0,1.0,95.0,25,98.0,0.38
1,Hadern,48.11476,11.48782,Entire home/apt,2,1.0,1.0,80.0,128,97.0,1.32
2,Berg am Laim,48.11923,11.63726,Entire home/apt,5,1.0,3.0,95.0,48,95.0,0.48
3,Maxvorstadt,48.15198,11.56486,Entire home/apt,4,1.0,1.0,120.0,82,97.0,0.81
4,Pasing-Obermenzing,48.13898,11.46612,Entire home/apt,3,1.0,1.0,55.0,30,93.0,0.30


In [19]:
df_listings_cleansed.dtypes

neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
room_type                  object
accommodates                int64
bedrooms                  float64
beds                      float64
price                     float64
number_of_reviews           int64
review_scores_rating      float64
reviews_per_month         float64
dtype: object

### price muunnos onnistui<br>
jatketaan datan tutkimista ja muokkaamista

In [20]:
#tutkitaan kuinka paljon löytyy tyhjiä soluja eri sarakkeissa
df_listings_cleansed.isnull().sum()

neighbourhood_cleansed       0
latitude                     0
longitude                    0
room_type                    0
accommodates                 0
bedrooms                    31
beds                        12
price                        0
number_of_reviews            0
review_scores_rating      4024
reviews_per_month         3784
dtype: int64

Ei mitään kovin vakavaa, poistetaan rivit joissa bedrooms tai bed on nolla ja korvataan review_scores_rating ja reviews_per_month null arvot arvolla 0.

In [34]:
#nollarivien poisto
df_listings_cleansed_zeros_removed =df_listings_cleansed.dropna(subset = ['bedrooms', 'beds'])
#Null arvojen korvaaminen 
df_listings_cleansed_zeros_removed_NaN_replaced = df_listings_cleansed_zeros_removed.fillna(0)

In [35]:
df_listings_cleansed_zeros_removed_NaN_replaced.describe()

,latitude,longitude,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month
count,12315.000000,12315.000000,12315.000000,12315.000000,12315.000000,12315.000000,12315.000000,12315.000000,12315.000000
mean,48.140001,11.564667,2.711571,1.091758,1.551929,119.205441,13.796346,64.097929,0.649449
std,0.025130,0.043892,1.515163,0.707182,1.237362,126.958472,37.342559,44.912452,1.354135
min,48.063700,11.378370,1.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000
25%,48.123285,11.540785,2.000000,1.000000,1.000000,55.000000,0.000000,0.000000,0.000000
50%,48.137410,11.565700,2.000000,1.000000,1.000000,89.000000,2.000000,93.000000,0.180000
75%,48.156120,11.584835,3.000000,1.000000,2.000000,140.000000,10.000000,100.000000,0.770000
max,48.231090,11.712280,16.000000,25.000000,50.000000,5000.000000,680.000000,100.000000,73.580000


price vaihtelu $8 ja $5000 välillä, hintaennustimen kohdalla ääriarvoja on ehkä syytä poistaa.<br>
Palataan siihen myöhemmin.

In [36]:
df_listings_cleansed_zeros_removed_NaN_replaced.isnull().sum()

neighbourhood_cleansed    0
latitude                  0
longitude                 0
room_type                 0
accommodates              0
bedrooms                  0
beds                      0
price                     0
number_of_reviews         0
review_scores_rating      0
reviews_per_month         0
dtype: int64

In [ ]:
#data vaikuttaa olevan ok siivousoperaation jäljiltä.

In [40]:
df_listings_cleansed_zeros_removed_NaN_replaced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12315 entries, 0 to 12356
Data columns (total 11 columns):
neighbourhood_cleansed    12315 non-null object
latitude                  12315 non-null float64
longitude                 12315 non-null float64
room_type                 12315 non-null object
accommodates              12315 non-null int64
bedrooms                  12315 non-null float64
beds                      12315 non-null float64
price                     12315 non-null float64
number_of_reviews         12315 non-null int64
review_scores_rating      12315 non-null float64
reviews_per_month         12315 non-null float64
dtypes: float64(7), int64(2), object(2)
memory usage: 1.1+ MB


Muokataan df_listings_reviews datasetin sarakkeet

In [25]:
cols2 = ['reviewer_id',
        'reviewer_name',
        'date',
        'comments',
        'listing_id',
        'neighbourhood_cleansed',
        'room_type',
        'reviews_per_month',
        'number_of_reviews',
        'review_scores_rating'
        ]

df_listings_reviews_cleansed = pd.read_csv('listings_reviews.csv', usecols=cols2)
#tarkistetaan tulos
df_listings_reviews_cleansed.head()

,neighbourhood_cleansed,room_type,number_of_reviews,review_scores_rating,reviews_per_month,listing_id,date,reviewer_id,reviewer_name,comments
0,Ludwigsvorstadt-Isarvorstadt,Entire home/apt,25,98.0,0.38,36720,2014-05-09,11840468,Mikhail,I would like to thank Gabriella as her apartme...
1,Ludwigsvorstadt-Isarvorstadt,Entire home/apt,25,98.0,0.38,36720,2014-05-20,5466213,Kim,Gabriela's place was absolutely fantastic. It...
2,Ludwigsvorstadt-Isarvorstadt,Entire home/apt,25,98.0,0.38,36720,2014-07-25,2062882,Juan R.,"Quiet place, open to a courtyard, with all the..."
3,Ludwigsvorstadt-Isarvorstadt,Entire home/apt,25,98.0,0.38,36720,2014-07-27,1225618,David,The best Airbnb expeierence I've had. The apar...
4,Ludwigsvorstadt-Isarvorstadt,Entire home/apt,25,98.0,0.38,36720,2014-09-13,13977301,Cal,"All first rate ! Beautiful apartment, comforta..."


In [42]:
df_listings_reviews_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170005 entries, 0 to 170004
Data columns (total 10 columns):
neighbourhood_cleansed    170005 non-null object
room_type                 170005 non-null object
number_of_reviews         170005 non-null int64
review_scores_rating      169696 non-null float64
reviews_per_month         170005 non-null float64
listing_id                170005 non-null int64
date                      170005 non-null object
reviewer_id               170005 non-null int64
reviewer_name             170005 non-null object
comments                  169940 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 13.0+ MB


In [43]:
df_listings_reviews_cleansed.isnull().sum()

neighbourhood_cleansed      0
room_type                   0
number_of_reviews           0
review_scores_rating      309
reviews_per_month           0
listing_id                  0
date                        0
reviewer_id                 0
reviewer_name               0
comments                   65
dtype: int64

In [ ]:
# Null arvot eivät haittaa joten annetaan olla alkuperäisessä muodossa.